In [ ]:
pip install conllu

In [ ]:

import pandas as pd
import io
from google.colab import drive
drive.mount('/content/drive')
import requests
from functools import partial
import csv
from dataclasses import dataclass
import pandas as pd
import conllu
from conllu import parse

In [ ]:

depth_df = pd.DataFrame(columns=['Sentence', 'Tree Depth'])
csv_filename = 'sentences_and_depths.csv'

def calculate_tree_depth(sentence):
    def get_depth(token_id, token_dict, depth=0):
        children = [t for t in sentence if t['head'] == token_id]
        if not children:
            return depth
        return max(get_depth(child['id'], token_dict, depth + 1) for child in children)

    # Create a dictionary for quick access to tokens by their id
    token_dict = {token['id']: token for token in sentence}

    # The root token has a 'head' value of 0, start from there
    return get_depth(0, token_dict)

Result_df = pd.DataFrame()

payload = {
    "data": "I Love NLP very much",
    "tokenizer": "ranges",
    "model": "english-atis-ud-2.12-230717",
    "tagger": True,
    "parser": True
}

filename = "/content/drive/MyDrive/NLP - Project/output.csv" #CSV input should only have one column called "Raw"


@dataclass
class Row:
    text: str


def get_csv_reader():
    return partial(csv.reader)


with open(filename, encoding="utf-8") as file:
    reader = get_csv_reader()
    for row in reader(file):
        try:
            r = Row(*row)
            payload = {
                "data": f"{r.text}",
                "tokenizer": "ranges",
                "model": "arabic-padt-ud-2.12-230717",
                "tagger": True,
                "parser": True,
            }
            response = requests.post("https://lindat.mff.cuni.cz/services/udpipe/api/process", data=payload)
            result = response.json().get("result", None)
            # print(result)
            Con = parse(result)
            # print(f"\n\nParse:", Con)
            for sentence in Con:
              depth = calculate_tree_depth(sentence)
              sentence_str = ' '.join([token['form'] for token in sentence])
              depth_df = depth_df.append({'Sentence': sentence_str, 'Tree Depth': depth}, ignore_index=True)
              depth_df.to_csv(csv_filename, index=False, encoding='utf-8')
              # print("Parse:", sentence)
              # print("Tree depth:", depth)
              # print("#" * 50, "\n")  # Separator for readability

            # Result_df = pd.DataFrame(list(Con))
            # print("Dataframe:", Result_df )
            # print("\n\n")
        except Exception as e:
            print(f"Operation got an Error: {e}")
            raise Exception


#Result_df.to_#csv('API-results.csv')